In [12]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

# temporary solution for relative imports in case sadl is not installed
# if sadl is installed, no need to use the line
import sys
import os
import inspect
from inspect import signature
import numpy as np
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

from SADL.static_data.algorithms import pyod

kwargs = {"algorithm": "cblof","contamination":2, "label_parser": True, "n_neighbors": 4, "method": 'method'}
model = pyod.PyodAnomalyDetection(**kwargs)

#print(model.algorithm)
print(model.label_parser)
print(model.model)
#print(signature(model.__class__).parameters.values())

kwargs = {"algorithm": "abod","contamination":2, "label_parser": False, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)
print(model.label_parser)
print(model.model)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ValueError: Invalid parameter 'n_neighbors' for estimator <SADL.static_data.algorithms.pyod.PyodAnomalyDetection object at 0x7f524a693450>.<class 'pyod.models.cblof.CBLOF'> Valid parameters are: {'label_parser': True, 'algorithm': 'cblof', 'alpha': 0.9, 'beta': 5, 'check_estimator': False, 'clustering_estimator': None, 'contamination': 0.1, 'n_clusters': 8, 'n_jobs': None, 'random_state': None, 'use_weights': False}.

Data generation

In [2]:
# Define the number of inliers and outliers
n_samples = 200
outliers_fraction = 0.25

# Compare given detectors under given settings
# Initialize the data
xx, yy = np.meshgrid(np.linspace(-7, 7, 100), np.linspace(-7, 7, 100))
n_inliers = int((1. - outliers_fraction) * n_samples)
n_outliers = int(outliers_fraction * n_samples)
ground_truth = np.zeros(n_samples, dtype=int)
ground_truth[-n_outliers:] = 1

# Show the statics of the data
print('Number of inliers: %i' % n_inliers)
print('Number of outliers: %i' % n_outliers)
print('Ground truth shape is {shape}. Outlier are 1 and inliers are 0.\n'.format(shape=ground_truth.shape))
print(ground_truth)

X1 = 0.3 * np.random.randn(n_inliers // 2, 2)
X2 = 0.3 * np.random.randn(n_inliers // 2, 2)
X = np.r_[X1, X2]
# Add outliers
X = np.r_[X, np.random.uniform(low=-6, high=6, size=(n_outliers, 2))]


Number of inliers: 150
Number of outliers: 50
Ground truth shape is (200,). Outlier are 1 and inliers are 0.

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


Model Fitting - errors


In [3]:
%load_ext autoreload
%autoreload 2
kwargs = {"algorithm": "abod","contamination":2, "label_parser": True, "n_neighbors": 4, "method": 'method'}
model = pyod.PyodAnomalyDetection(**kwargs)

# Fit the model using SADL Library - error en fit
model.fit(X)
kwargs = {"algorithm": "abod","contamination":0.1, "label_parser": True, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)
#print(model.model)
# Fit the model using SADL Library - sin error
#model.fit(X)
scores_pred = model.decision_function(X) * -1
print(scores_pred)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dict_items([('algorithm', 'abod'), ('contamination', 2), ('label_parser', True), ('n_neighbors', 4), ('method', 'method')])
PYODerror fit(): ('method', 'is not a valid method')
For further reference please see: https://pyod.readthedocs.io/en/latest/
dict_items([('algorithm', 'abod'), ('contamination', 0.1), ('label_parser', True), ('n_neighbors', 4), ('method', 'fast')])
PYODerror decision_function(): This ABOD instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.
For further reference please see: https://pyod.readthedocs.io/en/latest/


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

Model Fitting - OK


In [4]:
%load_ext autoreload
%autoreload 2
kwargs = {"algorithm": "abod","contamination":0.1, "label_parser": True, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.model)
# Fit the model using SADL Library - sin error
model.fit(X)
scores_pred = model.decision_function(X) * -1
print(scores_pred)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dict_items([('algorithm', 'abod'), ('contamination', 0.1), ('label_parser', True), ('n_neighbors', 4), ('method', 'fast')])
ABOD(contamination=0.1, method='fast', n_neighbors=4)
[1.09060105e+03 3.48289410e+05 9.85275621e+04 4.87820052e+04
 1.05325478e+02 1.31550893e+04 7.62993510e+02 4.20386945e+02
 2.90681025e+04 1.84217398e+04 9.57667552e+03 7.63688546e+05
 8.98452308e+03 6.59444095e+01 3.02500875e+04 1.62069932e+04
 3.57530910e+05 2.76798837e+01 7.06156608e+04 5.67778188e+04
 1.42101122e+04 4.24237556e+03 2.64157511e+03 2.32408820e+06
 2.87910491e+01 1.08781749e+05 1.06643332e+05 6.85402693e+01
 2.64359988e+02 1.00038532e+06 6.44583070e+03 2.32931655e+03
 3.76862245e+04 6.46578904e+02 5.15553402e+03 1.39765114e+05
 7.30887418e+04 1.37694839e+05 6.48635776e+03 1.11617935e+06
 8.15144812e+02 6.54518252e+03 1.27320724e+04 9.13278155e+04
 6.69633632e+03 6.45340282e+04 4.79888864e+03 5.08682525e+05
 8

Model Predict w/ label_parser function

In [35]:
%load_ext autoreload
%autoreload 2

def function(X):
    print("Estoy en funcion")
    return X < 0.5

kwargs = {"algorithm": "abod","contamination":0.1, "label_parser": function, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.get_params())
model.predict(X)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'label_parser': <function function at 0x7f9537a28900>, 'algorithm': 'ABOD', 'contamination': 0.1, 'method': 'fast', 'n_neighbors': 4}
Estoy en funcion


array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [False,  True],
       [ True,  True],
       [ True, False],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True, False],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True, False],
       [ True,  True],
       [ Tr

Model Predict wo/ label_parser

In [34]:
%load_ext autoreload
%autoreload 2


def function(X):
    print("Estoy en funcion")
    return X < 0.5

kwargs = {"algorithm": "abod","contamination":0.1, "label_parser": None, "n_neighbors": 4, "method": 'fast'}
model.set_params(**kwargs)

print(model.get_params().keys())
model.fit(X)
model.predict(X)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dict_keys(['label_parser', 'algorithm', 'contamination', 'method', 'n_neighbors'])


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1])

In [14]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))
from SADL.static_data.algorithms.pyod import PyodAnomalyDetection
kwargs = {"algorithm": "cblof","contamination":0.2}
model = PyodAnomalyDetection(**kwargs)

print(model.get_params())